In [ ]:
!unzip /content/RAVDESS_dataset -d /content/RAVDESS_dataset/

In [ ]:
!pip install tensorflow audiomentations

In [37]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
import numpy as np
import librosa
import os
import glob
import soundfile as sf
import audiomentations

sample_rate = 16000

In [38]:
# Function to extract features
def extract_features(file_path, mfcc=True, chroma=True, mel=True, contrast=True, tonnetz=True, zcr=True):
    with sf.SoundFile(file_path) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        if chroma:
            stft = np.abs(librosa.stft(X))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T, axis=0)
            result = np.hstack((result, mel))
        if contrast:
            contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T, axis=0)
            result = np.hstack((result, contrast))
        if tonnetz:
            tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T, axis=0)
            result = np.hstack((result, tonnetz))
        if zcr:
            zcr = np.mean(librosa.feature.zero_crossing_rate(y=X).T, axis=0)
            result = np.hstack((result, zcr))
    return result

# Augmentation function
def augment_data(x, y, sample_rate, aug_factor=2):
    augmented_data = []
    augmented_labels = []
    aug = audiomentations.Compose([
        audiomentations.AddGaussianNoise(p=0.5),
        audiomentations.TimeStretch(p=0.5),
        audiomentations.PitchShift(p=0.5),
    ])
    for i in range(len(x)):
        for _ in range(aug_factor):
            augmented_data.append(aug(x[i], sample_rate=sample_rate))
            augmented_labels.append(y[i])
    return np.array(augmented_data), np.array(augmented_labels)

# Function to load data and extract features
def load_data(dataset_path, focused_emotions, test_size=0.2):
    x, y = [], []
    for file in glob.glob(os.path.join(dataset_path, 'Actor_*', '*.wav')):
        file_name = os.path.basename(file)
        emotion = emotions[file_name.split("-")[2]]
        if emotion not in focused_emotions:
            continue
        feature = extract_features(file)
        x.append(feature)
        y.append(emotion)
    return np.array(x), np.array(y)

# Define the neural network architecture with dropout layers
def create_model(input_shape, num_classes, dropout_rate=0.5):
    model = Sequential([
        Dense(512, input_shape=input_shape, activation='relu'),
        Dropout(0.2),
        Dense(256, activation='relu'),
        Dropout(0.3),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax'),
        # Dense(num_classes, activation = 'sigmoid')
    ])
    return model

In [39]:
# Define dataset path, emotions, and focused_emotions
dataset_path = 'RAVDESS_dataset'
emotions = {'01':'neutral', '02':'calm', '03':'happy', '04':'sad', '05':'angry', '06':'fearful', '07':'disgust', '08':'surprised'}
focused_emotions = ['neutral', 'calm', 'happy', 'sad', 'disgust', 'angry']

# Load data and extract features
x, y = load_data(dataset_path, focused_emotions)

# # Apply data augmentation
# x_augmented = (x, y, sample_rate)

# # Combine original and augmented data
# x_combined = np.concatenate((x, x_augmented), axis=0)
# y_combined = np.concatenate((y, y), axis=0)

# Combine original and augmented data
x_combined = x
y_combined = y

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_combined)
num_classes = len(label_encoder.classes_)

# Shuffle and split the data into train and test sets
indices = np.arange(len(x_combined))
np.random.shuffle(indices)
x_shuffled = x_combined[indices]
y_shuffled = y_encoded[indices]
x_train, x_test = x_shuffled[:int(len(x_shuffled) * 0.8)], x_shuffled[int(len(x_shuffled) * 0.8):]
y_train, y_test = y_shuffled[:int(len(y_shuffled) * 0.8)], y_shuffled[int(len(y_shuffled) * 0.8):]

# Standardize features
scaler = StandardScaler()
x_train_combined_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

# Normalization
minmax_scaler = MinMaxScaler()
X_train_norm = minmax_scaler.fit_transform(x_train)
X_test_norm = minmax_scaler.transform(x_test)

In [46]:
# Define model parameters
input_shape = x_train.shape[1:]
dropout_rate = 0.5
learning_rate = 0.001
batch_size = 15
epochs = 300

# Create the model
model = create_model(input_shape, num_classes, dropout_rate)

# Compile the model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(learning_rate),
              metrics=['accuracy'])

# Train the model
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_split=0.2)
# # Train the model
# history = model.fit(x_train, y_train,
#                     batch_size=batch_size,
#                     epochs=epochs,
#                     validation_split=0.2)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

Epoch 1/300
45/45 [==============================] - 1s 10ms/step - loss: 16.7851 - accuracy: 0.1837 - val_loss: 2.0785 - val_accuracy: 0.1479
Epoch 2/300
45/45 [==============================] - 0s 7ms/step - loss: 4.8461 - accuracy: 0.1956 - val_loss: 1.7315 - val_accuracy: 0.2189
Epoch 3/300
45/45 [==============================] - 0s 6ms/step - loss: 3.1235 - accuracy: 0.1926 - val_loss: 1.7678 - val_accuracy: 0.1953
Epoch 4/300
45/45 [==============================] - 0s 7ms/step - loss: 2.3467 - accuracy: 0.1793 - val_loss: 1.7606 - val_accuracy: 0.2012
Epoch 5/300
45/45 [==============================] - 0s 6ms/step - loss: 2.0775 - accuracy: 0.1822 - val_loss: 1.7489 - val_accuracy: 0.2012
Epoch 6/300
45/45 [==============================] - 0s 6ms/step - loss: 1.9111 - accuracy: 0.2074 - val_loss: 1.7211 - val_accuracy: 0.2012
Epoch 7/300
45/45 [==============================] - 0s 6ms/step - loss: 1.8627 - accuracy: 0.2222 - val_loss: 1.7296 - val_accuracy: 0.2012
Epoch 8/300

In [47]:
# Predict for the test set
y_pred = model.predict(x_test)
y_pred

7/7 [==============================] - 0s 2ms/step


array([[6.4673154e-06, 9.3774909e-01, 3.8483654e-06, 2.3726780e-04,
        2.5408264e-02, 3.6595061e-02],
       [6.5219641e-02, 1.6546896e-01, 1.2830749e-01, 7.6425172e-02,
        4.2018044e-01, 1.4439827e-01],
       [4.2379006e-06, 8.0254990e-11, 4.5177811e-05, 9.9970174e-01,
        3.1247126e-08, 2.4877556e-04],
       ...,
       [3.3755805e-03, 2.2047974e-01, 1.0226637e-02, 2.4066877e-02,
        3.0823714e-01, 4.3361402e-01],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [1.0690881e-01, 3.9186362e-02, 6.7284308e-02, 1.5348046e-01,
        5.9797782e-01, 3.5162222e-02]], dtype=float32)